In [1]:
!pip install textblob
!pip install wordcloud
!python -m textblob.download_corpora
!pip install plotly
!pip install wordninja
!pip install pyspellchecker

  Using cached textblob-0.15.3-py2.py3-none-any.whl (636 kB)
  Using cached nltk-3.6.2-py3-none-any.whl (1.5 MB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached click-8.0.1-py3-none-any.whl (97 kB)
  Using cached tqdm-4.61.0-py2.py3-none-any.whl (75 kB)
  Using cached regex-2021.4.4-cp39-cp39-manylinux2014_x86_64.whl (730 kB)
  Using cached wordcloud-1.8.1-cp39-cp39-manylinux1_x86_64.whl (363 kB)
  Using cached Pillow-8.2.0-cp39-cp39-manylinux1_x86_64.whl (3.0 MB)
  Using cached matplotlib-3.4.2-cp39-cp39-manylinux1_x86_64.whl (10.3 MB)
  Using cached numpy-1.20.3-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.4 MB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached kiwisolver-1.3.1-cp39-cp39-manylinux1_x86_64.whl (1.2 MB)
[nltk_data] Downloading package brown to /home/daro/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/daro/nltk_data...
[nltk_data]   Package punkt 

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
from textblob import TextBlob
from wordcloud import WordCloud
import plotly.express as px